In [1]:
#from surprise import KNNWithZScore
#from surprise import Reader,Dataset
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
#from sklearn.cluster import KMeans

In [2]:
import nltk
from nltk.corpus import stopwords
import re
from nltk.stem import PorterStemmer

In [3]:
product=pd.read_csv('products - Sheet1.csv')
product.head(4)

,ProductId,Title,Image URL,Division,Category,Sub category,Article type,Offer price,Sale price,Color,Gender
0,E8AC41D107EA5DBB5DF561A684899B38,Color Dose Pearls Bronze 59 Nail Polish,"https://media6.ppl-media.com/tr:h-750,w-750,c-...",Makeup,Nails,Nail Polish,Nail Polish,59,59,Color Dose,women
1,04AE770377AABCCBCCDD5FBC93CC83E1,Rimmel Space Dust Nail Polish - Luna Love #004,"https://media6.ppl-media.com/tr:h-750,w-750,c-...",Makeup,Nails,Nail Polish,Nail Polish,310,310,Rimmel,women
2,21B0CF144B3CFFDDCCE4937C7731A7CE,Color Dose Pearls Copper Rust 58 Nail Polish,"https://media6.ppl-media.com/tr:h-750,w-750,c-...",Makeup,Nails,Nail Polish,Nail Polish,59,59,Color Dose,women
3,129CE0C7596D2D346E5CDC54E55A3A67,Color Dose Matt Nude 25 Nail Polish,"https://media6.ppl-media.com/tr:h-235,w-235,c-...",NaN,Nails,Nail Polish,Nail Polish,59,59,color dose,women


In [4]:
text_title=product.Title

In [7]:
stm=PorterStemmer()
timer=0
final_v1=pd.DataFrame(columns=['Text'])
for text in text_title:
    sent=text.lower()
    sent=nltk.word_tokenize(sent)
    fsent=''
    for words in sent:
        if words not in stopwords.words('english') and re.match('[a-z0-9 ]',words):
            words=stm.stem(words)
            fsent=fsent+words+' '
    timer=timer+1
    #print(7000-timer,' left')
    final_v1=final_v1.append({'Text':fsent},ignore_index=True)

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
vectorizer=CountVectorizer()
vectorizer.fit(final_v1.Text)
news_trans=vectorizer.transform(final_v1.Text)

In [13]:
text_df=pd.DataFrame(news_trans.toarray(),columns=vectorizer.get_feature_names())

In [14]:
product=product.drop(['Title','Image URL'],axis=1)

In [15]:
product.Division=product.Division.fillna(method='ffill')
product.Color=product.Color.fillna(method='ffill')

In [17]:
final_dum1=pd.get_dummies(product.drop('ProductId',axis=1))

In [18]:
final_dum_cp=final_dum1

In [19]:
last=pd.concat([final_dum_cp,text_df],axis=1)

In [25]:
last['ProductId']=product.ProductId

In [26]:
nbrs = NearestNeighbors(n_neighbors=10, algorithm='ball_tree').fit

In [63]:
#distances, indices = nbrs.kneighbors(final_dummies)
#prod_id=input("Enter id")
#valid=final_dummies[final_dummies.ProductId==prod_id]

In [27]:
prodId=input('Enter id ')
valid=last[last.ProductId==prodId]

Enter id Color Dose Pearls Bronze 59 Nail Polish


In [30]:
dist,indices=nbrs.(X=valid.drop('ProductId',axis=1),n_neighbors=10)

AttributeError: 'function' object has no attribute 'fit'

In [32]:
final_df=pd.DataFrame(columns=product.columns)
for ind in indices:
    final_df=final_df.append(product.iloc[ind])
    
    
    

In [33]:
final_df

,ProductId,Division,Category,Sub category,Article type,Offer price,Sale price,Color,Gender
0,E8AC41D107EA5DBB5DF561A684899B38,Makeup,Nails,Nail Polish,Nail Polish,59,59,Color Dose,women
2,21B0CF144B3CFFDDCCE4937C7731A7CE,Makeup,Nails,Nail Polish,Nail Polish,59,59,Color Dose,women
3,129CE0C7596D2D346E5CDC54E55A3A67,Makeup,Nails,Nail Polish,Nail Polish,59,59,color dose,women
3050,49A302A5541D8D81BBF75018BB7C5139,Makeup,Hair Care and Styling,Hair Color,Hair Color,60,60,khadi,women
6885,4418A925F29D1C784E63A1465DAAF1A8,Wellness,Condoms,Durex Condoms,Durex Condoms,60,60,Durex,women
6936,6C44B35677DA4F58F1CA6A0FDD755AAA,Body,Hand And Nail Care,Manicure,Manicure,60,60,Kaiv,women
6887,3F3BF37F84C089EBCBBE486463EA44CF,Wellness,Condoms,Durex Condoms,Durex Condoms,60,60,Durex,women
6878,789117106FB7064A80F7F15D70E64888,Wellness,Condoms,Durex Condoms,Durex Condoms,60,60,Durex,women
6893,F2812550583E52A3C95960A58B7AAA6C,Wellness,Condoms,Durex Condoms,Durex Condoms,60,60,Durex,women
1235,0E99130218B790AC7EC597235436CA99,Body,Soap,Handmade Soaps,Handmade Soaps,59,59,Mirah Belle,women


In [35]:
final_df.to_csv('top_10_product.csv')